In [59]:
#pip install pdfminer.six
#pip install doc2text
# pip install nltk
#pip install spacy
#python -m spacy download en_core_web_sm
# pip install nltk
#python -m nltk nltk.download('words')
#pip install constants

  Created wheel for constants: filename=constants-0.6.0-py3-none-any.whl size=5473 sha256=b98c50a6d54f3aa7fdc2d753e75a4d6b4a7eb58effb2d76e8bab7ae3622fe605
  Stored in directory: c:\users\anude\appdata\local\pip\cache\wheels\2c\f8\73\4eea26e0f7d0d170a29e16568c6d2d09bbbbdd169a42637afe
Successfully built constants


In [21]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from nltk.corpus import stopwords
import nltk
import constants as cs
import io
import json
import re
import spacy
import pandas as pd
from spacy.matcher import Matcher
from nltk.stem import WordNetLemmatizer


# enc_core_lg

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            converter.close()
            fake_file_handle.close()

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern])
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text


def extract_text(pdf_path):
    resume_text = ''
    for page in extract_text_from_pdf(pdf_path):
        resume_text += page
    return resume_text

# extract address
def extract_address(resume_text):
    nlp_text = nlp(resume_text)
    pattern = [{'LOWER': 'address'}, {'IS_DIGIT': True}]
    
    matcher.add('ADDRESS', [pattern])
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

def extract_education(resume_text):
    STOPWORDS = set(stopwords.words('english'))
    EDUCATION = ['BE','B.E.', 'B.E', 'BS','B.S','B.Com','BCA','ME','M.E', 'M.E.', 'M.S','B.com','10','10+2','BTECH', 'B.TECH', 'M.TECH', 'MTECH', 'SSC', 'HSC', 'C.B.S.E','CBSE','ICSE', 'X', 'XII','10th','12th',' 10th',' 12th','Bachelor of Arts in Mathematics','Master of Science in Analytics','Bachelor of Business Administration','Major: Business Management']


    nlp_text = nlp(resume_text)
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education


def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)

    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number


def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None


def extract_skills(resume_text):
    nlp = spacy.load('en_core_web_sm')
    noun_chunks = nlp(resume_text)
    nlp_text = nlp(resume_text)

    tokens = [token.text for token in nlp_text if not token.is_stop]

    data = pd.read_csv("skills.csv") 
    skills = list(data.columns.values)
    
    skillset = []
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
 
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]
               

def extract_dob(text):
    #TODO: add other regular expressions
    result1=re.findall(r"[\d]{1,2}/[\d]{1,2}/[\d]{4}",text)
    result2=re.findall(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}",text)           
    result3= re.findall(r"[\d]{1,2} [ADFJMNOSadfjmnos]\w* [\d]{4}",text)
    result4=re.findall(r"([\d]{1,2})\.([\d]{1,2})\.([\d]{4})",text)
                
    l=[result1,result2,result3,result4]
    for i in l:
        if i==[]:
            continue
        else:
            return i

def extract_experience(resume_text):
    wordnet_lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # word tokenization 
    word_tokens = nltk.word_tokenize(resume_text)

    # remove stop words and lemmatize  
    # filtered_sentence = [w for w in word_tokens if not w in stop_words and wordnet_lemmatizer.lemmatize(w) not in stop_words] 
    filtered_sentence = [w for w in word_tokens]
    sent = nltk.pos_tag(filtered_sentence)
    print(sent)

    # parse regex
    cp = nltk.RegexpParser('P: {<NNP>+}')
    cs = cp.parse(sent)

    ###TODO: afterwards with stanza - stanford's nlp , spacy-stanza
    test = []
    
    for vp in list(cs.subtrees(filter=lambda x: x.label()=='P')):
        test.append(" ".join([i[0] for i in vp.leaves() if len(vp.leaves()) >= 2]))

    # Search the word 'experience' in the chunk and then print out the text after it
    x = [x[x.lower().index('experience') + 10:] for i, x in enumerate(test) if x and 'experience' in x.lower()]
    return x

In [22]:
a = 'My skills include Python and Java'
extract_experience(a)

[('My', 'PRP$'), ('skills', 'NNS'), ('include', 'VBP'), ('Python', 'NNP'), ('and', 'CC'), ('Java', 'NNP')]


[]

In [3]:
file_path = 'D:/Coding --python/Resumes/atul sharma.pdf'
text = ''
page = ''
for page in extract_text_from_pdf(file_path):
    text += ' ' + page

nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

In [11]:
#TODO
sample_dob = 'October 11, 1961'
extract_dob(sample_dob)

In [4]:
dictionary ={
    "Name" : extract_name(text),
    "Mobile Number" : extract_mobile_number(text),
    "Email" : extract_email(text),
    "Skills" : extract_skills(text),
    # "Address" : extract_address(text),
    "Education" : extract_education(text),
    "Date of Birth" : extract_dob(text),
    "Experience" : extract_experience(text)
    # "Projects" : extract_projects(text),
    # "Achievements" : extract_achievements(text),
    # "Certifications" : extract_certifications(text),
    # "Languages" : extract_languages(text),
    # "References" : extract_references(text),
    # "Interests" : extract_interests(text),
    # "Additional Information" : extract_additional_information(text)
}
dictionary = {k:v for k,v in dictionary.items() if v}
print(dictionary)

{'Name': 'Atul Sharma', 'Mobile Number': '+919223339060', 'Email': 'Atuls61@gmail.com', 'Skills': ['Regulatory', 'Process', 'Benchmarking', 'Inventory', 'Vendors', 'Erp', 'Reporting', 'Oracle', 'Tax', 'Compliance', 'Accounting', 'Proposal', 'Banking', 'Operations', 'Strategy', 'Mis', 'Sourcing', 'Modeling', 'Analysis', 'Finance', 'Negotiation', 'Documentation', 'Sap', 'Budgeting', 'Controls'], 'Experience': [' February']}


In [5]:
json_object = json.dumps(dictionary, indent = 4)
with open("extracted_details1.json", "w") as outfile:
    outfile.write(json_object)
# print(extract_name(text))
# print(extract_mobile_number(text))
# print(extract_email(text))
# print(extract_dob(text))
# print(extract_skills(text))
# print(extract_experience(text))